<a href="https://colab.research.google.com/github/hsj51/HardSub_to_SoftSub/blob/main/HardSub_to_softsub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <img src='https://fiverr-res.cloudinary.com/images/t_main1,q_auto,f_auto,q_auto,f_auto/gigs/201283509/original/bded753530fef9a172ef7c932905e2496cd2072c/make-srt-and-ass-subtitle-file-from-any-hardsub-video.png' height="50" alt="iceWM" /> ___Hardsub2Softsub v1.0___
- Warning : For better result, please use TPU Mode. Othrewise, converting process may fail -->

In [ ]:
#@title ← Install Modules
#@markdown - Need only for one time, you can skip this step for another operation

#@markdown <br><b>Upload SA.zip or single SA file</b>
#@markdown - Select if you want to upload `sa.zip`(Only a single random file will be selected) or only single `sa-account.json` file.
#@markdown - For sa.zip make sure all sa files are in root dir of zip.
#@markdown - If uploading through gdrive link, it must be of the format `https://drive.google.com/uc?id=1QGMGef-QHuJAXsOtPyKwDz1WbS2ndGCl`

file_type = "sa.zip" #@param ["sa.zip","sa-account.json"]
file_from = "PC" #@param ["Gdrive", "PC", "Direct Link"]
url = "https://foo.bar.workers.dev/0:/sa.zip" #@param {type:"string"}


def get_sa():
  if file_from == "Gdrive":
    gdown.download(url, file_type, quiet=False)

  elif file_from == "PC":
    from google.colab import files
    uploaded = files.upload()
    for fn in uploaded.keys():
      os.rename(fn, file_type)
  else:
    os.system(f'wget {url} -O {file_type}')

  if file_type == 'sa.zip':
    os.system('unzip sa.zip -d accounts')
    os.system('rm sa.zip')
    current_file_name = random.choice(os.listdir("accounts"))
    os.rename('accounts/'+str(current_file_name), 'sa-account.json')
    os.system('rm -R accounts')

import os, urllib.request, random, gdown
from pathlib import Path
from IPython.display import clear_output

HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/hsj51/HardSub_to_SoftSub/master/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing Dependencies...", ty='twg')
os.chdir('/content')
ocr = "ocr"
if not Path(ocr).exists():
  os.mkdir(ocr)
os.chdir(ocr)
os.system('pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib')

os.system('pip install srtmerge')
os.system('wget https://raw.githubusercontent.com/hsj51/HardSub_to_SoftSub/master/main.py')
os.system('wget https://raw.githubusercontent.com/hsj51/HardSub_to_SoftSub/master/main2.py')
get_sa()
clear_output()
print("Install Finished")

In [ ]:
#@title ← Upload Images from Video Sub Finder for OCR
#@markdown - Please Upload RAR file contaning Images (RGB Images)
#@markdown - Gdrive Link format must be format as below (Otherwise downloaded file will not work)
#@markdown - <code>https://drive.google.com/u/6/uc?id=1QGMGef-QHuJAXsOtPyKwDz1WbS2ndGCl&export=download</code>


import gdown
Upload_from = "Gdrive" #@param ["Gdrive", "PC", "Direct Link"]
url = "https://drive.google.com/uc?id=1BXuLPELUKyAepK5_7PtcQWI-JzXZpZe8" #@param {type:"string"}
if Upload_from == "Gdrive":
  output = "RGBimages.rar"
  gdown.download(url, output, quiet=False)

elif Upload_from == "PC":
 from google.colab import files
 uploaded = files.upload()

else:
 os.system('pip install wget')
 import wget
 filename = wget.download(url)
 print(filename)
 print("Downloaded")
 
os.system('cp *.rar backup.bak')

#Codemaster V2.1 VSF  

#@title ← Extract RAR

import fnmatch,os

up = "./up"
down = "./down"
from pathlib import Path
if not Path('up').exists():
  os.mkdir(up)
if not Path('down').exists():
  os.mkdir(down)

os.system("unrar x *.rar up")
os.system("unrar x *.rar down")

In [ ]:
#@title ← Select your sample photo to crop
#@markdown - Please set <b>line between upper and lower </b>subtitle. To adjust fine tuning, please use keyboard arrow.
#@markdown - Enhancement (optional) for Better OCR reading.(<i>Uncheck if original movie is very dark and subtitle color is unclear</i>)

Enhancement = True  # @param {type:"boolean"}
#@markdown - Enhancement_value (Default = 0.5) 
Enhancement_value = 0.5 #@param {type:"slider", min:0.5, max:0.9, step:0.1}

if Enhancement:
  factor = Enhancement_value
else:
  factor = 1



#os.makedirs("./temp", exist_ok=True)
import os
#if not os.path.exists(path):
 #   os.makedirs(path)

%matplotlib inline
from PIL import Image, ImageEnhance
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np
import re
from IPython.display import display, clear_output
from ipywidgets import widgets
from os import listdir
from os.path import isfile, join
from IPython.display import HTML
from base64 import b64encode


mypath = 'up/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

if len(onlyfiles)>0:
  filename_ch = widgets.Dropdown(
      options=onlyfiles,
      description='Choose file:',
  )
  butt = widgets.Button(description='Select')
  def on_butt_clicked(b):
      print(filename_ch.value)
      
      source = str(mypath)+filename_ch.value
      im = Image.open(source)
      #display(im)
      #f = open("./temp/selectedvideo.txt","w") 
      #f.write(filename_ch.value)
      #f.close() 
      w, h = im.size
      print("Image Resolution "+str(w)+"x"+str(h))

      def f(crop_line):
          plt.figure(figsize = (20,6))
          image = plt.imread(source)
          plt.imshow(image)
          x = np.linspace(crop_line, crop_line, num=w)
          plt.plot(x)
          #plt.ylim(0, 2000)
          plt.show()
      interactive_plot = interactive(f, crop_line=(1, h))
      output = interactive_plot.children[-1]
      output.layout.height = '{h}px'
      interactive_plot   

      def on_button_clicked(b):
           with output:
        
            s = str(interactive_plot)
            pattern = "value=(.*?),"
            substring = re.search(pattern,s).group(1)
            z = int(substring)
            #print(substring)
            print('Please wait.. Images are being cropped')
            f = r'up'
            for file in os.listdir(f):
              f_img = f+"/"+file
              img = Image.open(f_img)
              img = img.crop((0,0,w,z))
              enhancer = ImageEnhance.Brightness(img)
              im_output = enhancer.enhance(factor)
              im_output.save(f_img)
            g = r'down'
            for file in os.listdir(g):
              g_img = g+"/"+file
              img = Image.open(g_img)
              img = img.crop((0,z,w,h))
              enhancer = ImageEnhance.Brightness(img)
              im_output = enhancer.enhance(factor)
              im_output.save(g_img)
            #clear_output()
            print('Image_Cropped')
            if Enhancement:
              print('Image_Enhanced')
              
            #im1 = im.crop((0, 0, w, z))
            #im2 = im.crop((0, z, w, h))
            #display(im1)
            #display(im2)

      button = widgets.Button(description="Crop")
      output = widgets.Output()
      display(interactive_plot, button, output)
      button.on_click(on_button_clicked)
      
  butt.on_click(on_butt_clicked)
  # display
  display(widgets.VBox([filename_ch,butt]))
else:
  print('Upload photo at first')

In [ ]:
#@markdown <h3>←  Extract files </h3>

import fnmatch,os
#temp = "./temp"

#from pathlib import Path
#if not Path('temp').exists():
#  os.mkdir(temp)
#os.system("unrar x *.rar temp")
#Counting files
path, dirs, files = next(os.walk("up"))
file_count = len(files)

#Additional Speed Up
float_num = file_count/15
x = round(float_num)
i = x + 1

#Reading Data
ext = ".jpg"
counter = 0
dir_counter = 0
os.mkdir(f"images{dir_counter}")
current_dir = f"images{dir_counter}"
for file in os.listdir(up):
    if counter % i == 0:
        dir_counter +=1
        os.mkdir(f"images{dir_counter}")
        current_dir = f"images{dir_counter}"
    if file.endswith(tuple(ext)):
      os.rename(f"{up}/{file}",f"{current_dir}/{file}")
      counter += 1
# write counter
print(f"Total {counter} {ext} files (Upper Line)")
with open('counter.txt','w') as f:
  f.write(str(counter))

#clean
os.rmdir(up)

#--------------------

#temp = "./temp"

#from pathlib import Path
#if not Path('temp').exists():
#  os.mkdir(temp)
#os.system("unrar x *.rar temp")
#Counting files
path, dirs, files = next(os.walk("down"))
file_count = len(files)

#Additional Speed Up
float_num = file_count/15
x = round(float_num)
i = x + 1

#Reading Data
ext = ".jpg"
counter = 0
dir_counter = 16
os.mkdir(f"images{dir_counter}")
current_dir = f"images{dir_counter}"
for file in os.listdir(down):
    if counter % i == 0:
        dir_counter +=1
        os.mkdir(f"images{dir_counter}")
        current_dir = f"images{dir_counter}"
    if file.endswith(tuple(ext)):
      os.rename(f"{down}/{file}",f"{current_dir}/{file}")
      counter += 1
# write counter
print(f"Total {counter} {ext} files (Lower Line)")
with open('counter.txt','w') as f:
  f.write(str(counter))

#clean
os.rmdir(down)



         

         

In [ ]:
 #@title ← RUN (Upper Line)
#@markdown - If progress freeze/hang, click this button again to resume OCR process.

import multiprocessing


def mymain(mydir):
  !python main.py {mydir}

for i in range(1,16):
    myimages = f"images{i}"
    p = multiprocessing.Process(target=mymain, args=(myimages,))
    p.start()
    
print("Done")
#Codemaster V2.2 VSF 

In [ ]:
 #@title ← RUN (Lower Line)
#@markdown - If progress freeze/hang, click this button again to resume OCR process.

import multiprocessing


def mymain(mydir):
  !python main2.py {mydir}

for i in range(17,32):
    myimages = f"images{i}"
    p = multiprocessing.Process(target=mymain, args=(myimages,))
    p.start()
    
print("Done")
#Codemaster V2.2 VSF 

In [ ]:
#@title ← Check Progress
#@markdown - If progress freeze/Hang, please re-click <b>"Run"</b> button, <u>wait a minute</u> and re-check progress.

import os
from IPython.display import HTML, display
import time

Counter = int(open("counter.txt","r").readline())

if os.path.isdir('texts'):
 file_count = len(os.listdir("texts"))

 print("Total File Done (Upper Line)")
 print(str(file_count)+" of "+str(Counter))
 if file_count == Counter:
  print("Upper Line OCR Completed")
  #!mv subtitle_output.srt up.srt

else:
 print('Initializing... Please be patient')


#  from google.colab import files
#  files.download('subtitle_output.srt') 

#while file_count != Counter:


def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=file_count, max=Counter))

out = display(progress(0, 100), display_id=True)

#--------------------------

if os.path.isdir('texts2'):
 file_count = len(os.listdir("texts2"))

 print("Total File Done (Lower Line)")
 print(str(file_count)+" of "+str(Counter))
 if file_count == Counter:
  print("Lower Line OCR Completed")
  #!mv subtitle_output.srt down.srt

else:
 print('Initializing... Please be patient')



#  from google.colab import files
#  files.download('subtitle_output.srt') 

#while file_count != Counter:


def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=file_count, max=Counter))

out = display(progress(0, 100), display_id=True)



In [ ]:
#@title ← Download Complete Subtitle
!srtmerge up.srt down.srt subtitle_output.srt
from google.colab import files
files.download('subtitle_output.srt') 

In [ ]:
#@title ← Clear All data for another operation
#@markdown - After that you can start from <u><b>step-2 (Upload RAR)</u></b> for another operation
Clear_backup_file = True  # @param {type:"boolean"}
!rm *.srt
!rm *.rar
!rm *.txt
!rm -R -- */
#!mkdir raw_texts
#!mkdir texts

if Clear_backup_file == True:
  !rm *.bak
!kill $(ps aux | grep 'main.py' | awk '{print $2}')
!kill $(ps aux | grep 'main2.py' | awk '{print $2}')

#Codemaster V2.2 VSF  